In [ ]:
import numpy as np
import torch
import torchvision
import tqdm
from torch import nn
from torch.nn import functional as F
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip '/content/drive/MyDrive/AcneDataset/archive.zip' -d '/content'

In [ ]:
from torchvision.transforms import Compose, Normalize, Resize, RandomRotation, RandomHorizontalFlip, ToTensor
train_transform = Compose(
        [
            Resize((256, 256)),
            RandomRotation(degrees= 30),
            RandomHorizontalFlip(),
            ToTensor(), 
            Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), 
        ]
    )

val_transform = Compose(
        [
            Resize((256, 256)),
            ToTensor(), 
            Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)), 
        ]
    )

train_dataset = torchvision.datasets.ImageFolder("drive/MyDrive/DatasetAcne/Dataset/train", transform=train_transform)
val_dataset = torchvision.datasets.ImageFolder("drive/MyDrive/DatasetAcne/Dataset/test", transform=val_transform)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size= 4, shuffle = True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size= 4, shuffle = False, drop_last= False)

In [ ]:
def train_one_epoch(model, train_dataloader, criterion, optimizer, device="cuda:0"):
    model = model.to(device).train()
    total_loss = 0
    num_batches = 0
    all_losses = np.array([])
    total_labels = np.array([])
    total_predictions = np.array([])

    for images, labels in tqdm(train_dataloader):
      images = images.to(device)
      labels = labels.to(device)
      predicted = model(images)
      loss = criterion(predicted, labels)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
      total_predictions = np.append(total_predictions, predicted.argmax(1).cpu().detach().numpy())
      total_labels = np.append(total_labels, labels.cpu().detach().numpy())
      total_loss += loss.item()
      num_batches += 1

    print(f"Train loss: {total_loss / num_batches}")
    print(f"Train accuracy: {(total_predictions == total_labels).mean()}")

def predict(model, val_dataloder, criterion, device="cuda:0"):
    model = model.to(device).eval()
    total_loss = 0
    total_labels = []
    num_batches = 0
    total_predictions = np.array([])
    total_labels = np.array([])

    with torch.no_grad():
      for images, labels in val_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        predicted = model(images)
        loss = criterion(predicted, labels)
        accuracy = (predicted.argmax(1) == labels).float().mean()
        total_predictions = np.append(total_predictions, predicted.argmax(1).cpu().detach().numpy())
        total_labels = np.append(total_labels, labels.cpu().detach().numpy())
        total_loss += loss.item()
        num_batches += 1
    
      print(f"Val loss: {total_loss / num_batches}")
      print(f"Val accuracy: {(total_predictions == total_labels).mean()}")
      losses = total_loss / num_batches
      predicted_classes = total_predictions
      true_classes = total_labels

    return losses, predicted_classes, true_classes


def train(model, train_dataloader, val_dataloader, criterion, optimizer, device="cuda:0", n_epochs=10):
    model.to(device)
    for epoch in range(n_epochs):
        print(f"Epoch number {epoch}")
        train_one_epoch(model, train_dataloader, criterion, optimizer, device)
        all_losses, predicted_labels, true_labels = predict(model, val_dataloader, criterion, device)

In [ ]:
model = torchvision.models.vgg13()

optimizer = torch.optim.Adam(model.parameters(), lr = 3e-4)
criterion = nn.CrossEntropyLoss()
n_epochs = 20
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
model

In [ ]:
model.classifier[6] = nn.Linear(in_features=4096, out_features=7, bias=True)

In [ ]:
import random
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [ ]:
set_random_seed(41)
train(model, train_dataloader, val_dataloader, criterion, optimizer, device, n_epochs)

In [ ]:
torch.save(model.state_dict(), 'Model/model.pth')